# Attribute Information: 
## (classes: edible=e, poisonous=p)

    cap-shape: bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s

    cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s

    cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y

    bruises: bruises=t,no=f

    odor: almond=a,anise=l,creosote=c,fishy=y,foul=f,musty=m,none=n,pungent=p,spicy=s

    gill-attachment: attached=a,descending=d,free=f,notched=n

    gill-spacing: close=c,crowded=w,distant=d

    gill-size: broad=b,narrow=n

    gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y

    stalk-shape: enlarging=e,tapering=t

    stalk-root: bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r,missing=?

    stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s

    stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s

    stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y

    stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y

    veil-type: partial=p,universal=u

    veil-color: brown=n,orange=o,white=w,yellow=y

    ring-number: none=n,one=o,two=t

    ring-type: cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z

    spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y

    population: abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y

    habitat: grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d


In [96]:
import pandas as pd
import numpy as np
import matplotlib
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
%matplotlib inline

In [97]:
md = pd.read_csv('~/Desktop/Machine Learning/datasets/mushrooms.csv',index_col=False)
md.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [98]:
def encode_df(df):
    mod = df.copy()
    column_list = list(mod)
    category_list = []
    for step in range(len(column_list)):
        category_list.append(mod[column_list[step]].unique()) 
        map_to_int = {name: n for n, name in enumerate(category_list[step])}
        mod['target'+str(step)] = mod[column_list[step]].replace(map_to_int)
    return mod, category_list

In [99]:
features=list(md)
del features[0]

In [100]:
c = np.asarray(list(md))

In [101]:
pd.set_option('display.max_colwidth', 10000)

In [112]:
var = []
for step in range(1,23):
    var.append('target'+str(step))
ed, lst = encode_df(md)
for x in range(len(lst)):
    lst[x] = list(enumerate(lst[x]))
df= pd.DataFrame({'feature': c,'mappings':lst})
attb = lst.copy()
del attb[0]


22

In [116]:
def encode_test(strVec, mapping):
    keys = []
    lstVec = list(strVec)
    if len(mapping) != len(lstVec):
        print ("Check length of string, mismatched")
    else:
        for i in range(len(strVec)):
            dd = dict(mapping[i])
            key = next(key for key, value in dd.items() if value == lstVec[i])
            keys.append(key)
    return keys
encode_test('kyefsfcnbt?kkwwpwoewvl',attb)

[4, 1, 4, 1, 7, 0, 0, 0, 8, 1, 4, 2, 3, 0, 0, 0, 0, 0, 1, 4, 3, 6]

In [104]:
X = ed[var]
y = ed['target0']
ed.head()
t = var
t.insert(0,'class')


In [105]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.4, random_state=0)

In [106]:
dt = DecisionTreeClassifier(min_samples_split=20, random_state=99)

In [107]:
dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=20, min_weight_fraction_leaf=0.0,
            presort=False, random_state=99, splitter='best')

In [138]:
# function takes in a string of length22 (without spaces or delimiters) for 
#the descriptors of each attibute in order and outputs prediction
#May need modification to parse matricies of samples
def predict_mushroom(strChar):
    x = encode_test(strChar, attb)
    ttout = dt.predict(np.asarray(x).reshape(1,-1))
    mp = dict(lst[0])
    return [mp[k] for k in ttout]
predict_mushroom('kyefsfcnbt?kkwwpwoewvl')

['p']

In [ ]:
def visualize_tree(tree, feature_names):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")

![title](outfile.jpg)

In [ ]:
visualize_tree(dt, features)

In [ ]:
predictions = dt.predict(X_test)

In [139]:
sk.metrics.accuracy_score(y_test, predictions, normalize=True, sample_weight=None)

1.0